<a href="https://colab.research.google.com/github/Haz3-jolt/Pycon-Algo-Workshop/blob/main/Pycon_AlgoTrading_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python-Powered Algorithmic Trading: From Theory to Practice

## Introduction


### Algorithmic Trading

Algorithmic trading uses computer algorithms to automatically execute trades based on predefined criteria, such as price, volume, or timing.

It aims to optimize trading strategies, minimize human error, and capitalize on market opportunities quickly.

By analyzing vast amounts of data and executing trades at high speeds, algo trading enhances efficiency and can lead to more precise and profitable trading decisions.


## Installation

### Configurations

In [1]:
# Set Configurations

# Matplotlib configuration
%matplotlib inline

# Suppress future warning messages
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

### TA-Lib C Module

[TA-Lib](https://ta-lib.org/) (Technical Analysis Library) is widely used by developers in finance and trading for performing technical analysis on market data.

- Provides over 200+ indicators such as ADX, MACD, RSI, Stochastic, Bollinger Bands, and more.
- Supports candlestick pattern recognition for identifying trading signals.
- Released in 2001, TA-Lib implements well-established algorithms that are still popular over 20 years later.
- Provides Open-source API for Python, but requires installation of a C module dependency for proper functionality.

Installation:
To install the required C module for TA-Lib, specific commands are needed.
For installation in Google Colab, refer to a [Stackoverflow answer](https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab) that provides step-by-step guidance.

In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb

Selecting previously unselected package libta-lib0.
(Reading database ... 123599 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sb

### TA-Lib Python Package

The original Python bindings for TA-Lib were created using **SWIG** (Simplified Wrapper and Interface Generator). However, these bindings can be challenging to install and are not as optimized in terms of performance as they could be.

To address these limitations, we will use a different [Python package](https://github.com/TA-Lib/ta-lib-python) that serves as a **Cython-based wrapper** for TA-Lib. Cython provides a more streamlined and efficient way to interface with the underlying C library, making it easier to install and faster in execution compared to the SWIG-based implementation.

This alternative package ensures better performance and ease of use, making it a preferred choice for developers working with TA-Lib in Python.

In [3]:
!pip install TA-Lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for TA-Lib: filename=TA_Lib-0.4.32-cp310-cp310-linux_x86_64.whl size=2063399 sha256=1c27d847cbe85c77eac2cb7199b848bec42c870d6e179fa6872b3d7e3ac83374
  Stored in directory: /root/.cache/pip/wheels/c3/21/bd/ca95eb09997c2a18fce271b98b10ffa9fcafbaa161be864dd7
Successfully built TA-Lib


### Yfinance
[yfinance](https://github.com/ranaroussi/yfinance) provides a Pythonic and efficient way to download market data, making it easy to access historical and real-time financial information.

In [4]:
!pip install yfinance

### Backtrader
[Backtrader](https://github.com/mementum/backtrader) is a popular open-source Python library designed for backtesting trading strategies.

It allows traders and developers to test and analyze their trading ideas using historical data before applying them in real-world markets.

In [5]:
!pip install backtrader
!pip install backtrader[plotting]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 9.6 MB/s eta 0:00:00


### Import Packages

In [6]:
import talib
import yfinance
import backtrader as bt
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Get Instrument Data

In [7]:
def get_instrument_data(symbol: str, period="1mo", interval="1d") -> pd.DataFrame:
    """
    Fetches historical instrument data for a given symbol within a specified period.

    Args:
        symbol (str): The instrument ticker symbol (e.g., 'AAPL' for Apple).
        period (str): Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        interval (str): Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                        Intraday data cannot extend last 60 days

    Returns:
        pd.DataFrame: DataFrame with columns: 'Open', 'Close', 'High', 'Low' and 'Volume'.
    """
    # Create a Ticker object for the given symbol
    ticker = yfinance.Ticker(symbol)

    # Fetch historical data
    instrument_data_df = ticker.history(period, interval, auto_adjust=False,
                                        rounding=True, actions=False)

    return instrument_data_df


In [8]:
data = get_instrument_data("^NSEI", "5y", "1d")

### Ticker Symbols


| Category     | Instrument   | Symbol      |
|--------------|--------------|-------------|
| Stock        | RELIANCE     | RELIANCE.NS |
|              | HDFC         | HDFCBANK.NS |
|              |              |             |
| Index        | Nifty50      | ^NSEI       |
|              | Bank Nifty   | ^NSEBANK    |
|              |              |             |
| Crypto       | Bitcoin      | BTC-INR     |
|              | Ethereum     | ETH-INR     |
|              |              |             |


You can search for the symbol of any instrument on [Yahoo's website](https://finance.yahoo.com/lookup/).


### Preview Stock Data

In [9]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-09-16 00:00:00+05:30,25406.65,25445.70,25336.20,25383.75,25383.75,168700
2024-09-17 00:00:00+05:30,25416.90,25441.65,25352.25,25418.55,25418.55,216000
2024-09-18 00:00:00+05:30,25402.40,25482.20,25285.55,25377.55,25377.55,215700
2024-09-19 00:00:00+05:30,25487.05,25611.95,25376.05,25415.80,25415.80,314500
2024-09-20 00:00:00+05:30,25525.95,25726.65,25426.60,25702.30,25702.30,0


## Visualize Stock Data

### Line Chart
A line chart displays the stock's closing price over time, offering a clear view of overall trends. It is a simple yet effective way to track price movements and identify upward or downward trends.

In [10]:
def display_line_chart(data):
  fig = px.line(data, x=data.index, y="Close")
  fig.show()

In [11]:
display_line_chart(data)

### Candlestick Chart
A candlestick chart visually represents the stock's open, high, low, and close prices for each time period. It is commonly used in technical analysis to detect patterns and trends in market behavior.

In [12]:
def display_candlestick_chart(data):
  candlestick_data = go.Candlestick(x=data.index,
                                  open=data['Open'],
                                  high=data['High'],
                                  low=data['Low'],
                                  close=data['Close'])
  fig = go.Figure(data=[candlestick_data])
  fig.show()

In [13]:
display_candlestick_chart(data)

## Instantiating Cerebro
**Cerebro** is the cornerstone of backtrader because it serves as a central point for:

- Gathering all inputs (Data Feeds),
- Gathering actors (Stratgegies),
- Gathering critics (Analyzers)
- Execute the backtesting/or live data feeding/trading
- Returning the results

## Understanding BackTrader Strategy

### Base Strategy

In [14]:
class BaseStrategy(bt.Strategy):
    def __init__(self):
        super().__init__()
        self.order = None

    def log(self, txt, dt=None):
        """Logging function for strategy"""
        dt = dt or self.data.datetime.datetime(0)
        print(f'{dt.isoformat()}: {txt}')


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY  Executed, {order.executed.price:.2f}')
                pass
            elif order.issell():
                self.log(f'SELL Executed, {order.executed.price:.2f}')
                pass
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'Order failed({order.getstatusname()})')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log("Trade Summary, gross: %.2f, net: %.2f, duration: %d\n" % (trade.pnl, trade.pnlcomm, trade.barlen))
        # self.log("Trade Summary, gross: %.2f, duration: %d\n" % (trade.pnl, trade.barlen))

### Log Close Price Strategy

In [15]:
class LogCloseStrategy(BaseStrategy):
    def next(self):
        # Simply log the closing price
        self.log('Close, %.2f' % self.data.close[0])

### Simple Buy Strategy

The Simple Buy Strategy buys a stock when its price falls for three consecutive sessions.

In [16]:
class SimpleBuyStrategy(BaseStrategy):
    def next(self):
        self.log('Close, %.2f' % self.data.close[0])

        if self.data.close[0] < self.data.close[-1]:
            if self.data.close[-1] < self.data.close[-2]:
                # price has been falling 3 sessions in a row
                self.log('BUY Triggered, %.2f' % self.data.close[0])
                self.buy()

### Simple Trade Strategy

The Simple Trade Strategy buys a stock when its price falls for three consecutive sessions and sells it after holding for n sessions.

In [17]:
class SimpleTradeStrategy(BaseStrategy):
    params = (('exitbars', 5),)

    def next(self):
        # self.log('Close, %.2f' % self.data.close[0])

        if not self.position:
            if self.data.close[0] < self.data.close[-1]:
                if self.data.close[-1] < self.data.close[-2]:
                    # price has been falling 3 sessions in a row
                    self.buy()
        else:
            if len(self) >= (self.bar_executed + self.params.exitbars):
                self.sell()

In [18]:
# Initialize Engine
cerebro = bt.Cerebro()

# Initialize Broker Configs
cerebro.broker.setcash(100000.0)  # Starting with 1 Lakh cash
cerebro.broker.setcommission(0.0003)  # 0.03% Brokerage


# Feed Data
data0 = bt.feeds.PandasData(dataname=data, tz=pytz.timezone('Asia/Kolkata'))
cerebro.adddata(data0)

# Add Strategy
cerebro.addstrategy(SimpleTradeStrategy, exitbars=3)

# Add Trade Analyzer
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name = "ta")

# Backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2019-09-26T00:00:00: BUY  Executed, 11469.85
2019-10-03T00:00:00: SELL Executed, 11322.25
2019-10-03T00:00:00: Trade Summary, gross: -147.60, net: -154.44, duration: 4

2019-10-04T00:00:00: BUY  Executed, 11388.45
2019-10-11T00:00:00: SELL Executed, 11257.70
2019-10-11T00:00:00: Trade Summary, gross: -130.75, net: -137.54, duration: 4

2019-11-25T00:00:00: BUY  Executed, 11922.45
2019-11-29T00:00:00: SELL Executed, 12146.20
2019-11-29T00:00:00: Trade Summary, gross: 223.75, net: 216.53, duration: 4

2019-12-03T00:00:00: BUY  Executed, 12067.65
2019-12-09T00:00:00: SELL Executed, 11939.10
2019-12-09T00:00:00: Trade Summary, gross: -128.55, net: -135.75, duration: 4

2019-12-26T00:00:00: BUY  Executed, 12211.85
2020-01-01T00:00:00: SELL Executed, 12202.15
2020-01-01T00:00:00: Trade Summary, gross: -9.70, net: -17.02, duration: 4

2020-01-07T00:00:00: BUY  Executed, 12079.10
2020-01-13T00:00:00: SELL Executed, 12296.70
2020-01-13T00:00:00: Trade Summary

## Analyze Trading Strategy

In [19]:
def display_trade_analysis(stats):
    """
    Displays a summary of trade analysis based on Backtrader's Analyzer results.

    This function prints out key statistics such as the number of open trades, closed trades, win/loss ratio,
    longest winning/losing streak, and overall net profit/loss. It also calculates and displays the strike rate
    (percentage of won trades out of total closed trades).

    :param stats: Backtrader's TradeAnalyzer object that holds trade performance metrics.
    """
    try:
        # Retrieve key metrics from the trade statistics
        total_open = stats.total.open       # Number of trades still open
        total_closed = stats.total.closed   # Number of trades that have been closed
        total_won = stats.won.total         # Total number of winning trades
        total_lost = stats.lost.total       # Total number of losing trades
        win_streak = stats.streak.won.longest   # Longest winning streak
        lost_streak = stats.streak.lost.longest # Longest losing streak
        pnl_net = round(stats.pnl.net.total, 2) # Net profit/loss, rounded to 2 decimal places

        # Calculate strike rate (percentage of closed trades that were winners)
        strike_rate = round((total_won / total_closed) * 100, 2) if total_closed > 0 else 0

        # Define headers and values for display
        headers_1 = ["Total Open", "Total Closed", "Total Won", "Total Lost"]
        headers_2 = ["Strike Rate(%)", "Win Streak", "Loss Streak", "Net PnL"]
        row_1 = [total_open, total_closed, total_won, total_lost]
        row_2 = [strike_rate, win_streak, lost_streak, pnl_net]

        # Determine the longest header length for formatting consistency
        max_header_length = max(len(headers_1), len(headers_2))

        # Prepare rows for display
        rows_to_display = [headers_1, row_1, headers_2, row_2]

        # Define the formatting for the display (20 characters width for each column)
        row_format = "{:<20}" * (max_header_length + 1)

        # Display the results
        print("\nTrade Analysis Results:")
        print("-" * 100)
        for row in rows_to_display:
            print(row_format.format("", *row))
        print("-" * 100)

    except (KeyError, AttributeError) as e:
        # Handle cases where no trade data is available or stats fields are missing
        print("No trade analysis available. It seems no trades have been taken or the stats are incomplete.")
        print(f"Error details: {e}")


### Display Analysis Result

In [21]:
# To print the full analysis (raw data)
# result[0].analyzers.ta.print()

# To display a clean, summarized format of the analysis
display_trade_analysis(result[0].analyzers.ta.get_analysis())


Trade Analysis Results:
----------------------------------------------------------------------------------------------------
                    Total Open          Total Closed        Total Won           Total Lost          
                    0                   121                 70                  51                  
                    Strike Rate(%)      Win Streak          Loss Streak         Net PnL             
                    57.85               8                   4                   2617.47             
----------------------------------------------------------------------------------------------------


## Implementing Practical Trade Strategies

### Simple Moving Average Crossover Strategy

Traders often use moving averages to identify trends and potential entry/exit points. A basic moving average strategy might involve buying when the price moves above a moving average and selling when it falls below


A moving average crossover system is an improvisation over the plain vanilla moving average system. Instead of the usual single moving average in a MA crossover system, the trader combines two moving averages.

The entry and exit rules for the crossover system is as stated below:

1. Buy (fresh long) when the short term moving averages turns greater than the long term moving average. Stay in the trade as long as this condition is satisfied

2. Exit the long position (square off) when the short term moving average turns lesser than the longer-term moving average

In [22]:
class SmaCross(BaseStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.params.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.params.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log(f'Buy : %.2f' % (self.data[0]))
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log(f'Sell: %.2f' % (self.data[0]))
            self.close()  # close long position

### Exponential Moving Average Crossover Strategy

An exponential moving average (EMA) scales the data according to its newness. Recent data gets the maximum weightage, and the oldest gets the least weightage.

In a crossover system, the price chart is overlaid with two EMAs. The shorter EMA is faster to react, while the longer EMA is slower to react.

The outlook turns bullish when the faster EMA crosses and is above the slower EMA. Hence one should look at buying the stock. The trade lasts upto a point where the faster EMA starts going below, the slower EMA

In [23]:
class EmaCross(BaseStrategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=7,  # period for the fast moving average
        pslow=21   # period for the slow moving average
    )

    def __init__(self):
        ema1 = bt.ind.EMA(period=self.params.pfast)  # fast moving average
        ema2 = bt.ind.EMA(period=self.params.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(ema1, ema2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log(f'Buy : %.2f' % (self.data[0]))
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log(f'Sell: %.2f' % (self.data[0]))
            self.close()  # close long position

### Relative strength Index Strategy
Relative strength Index or just RSI is the most popular leading indicator, which gives out the strongest signals during the periods of sideways and non-trending ranges.

RSI is a momentum oscillator which oscillates between 0 and 100 level.

A value between 0 and 30 is considered oversold. Hence the trader should look at buying opportunities.

A value between 70 and 100 is considered overbought. Hence the trader should look at selling opportunities.

In [24]:
class RSI(BaseStrategy):
    # List of parameters which are configurable for the strategy
    params = dict(
        period=14,            # Period for the RSI calculation
        rsi_overbought=60,    # Upper threshold for RSI (overbought)
        rsi_oversold=40       # Lower threshold for RSI (oversold)
    )

    def __init__(self):
        # Initialize the RSI indicator
        self.rsi = bt.ind.RSI(period=self.params.period)

    def next(self):
        # self.log(self.rsi[0])
        if not self.position:
            if self.rsi[0] < self.params.rsi_oversold:
                self.log(f'Buy : %.2f' % (self.data[0]))
                self.buy()  # enter long
        elif self.rsi[0] > self.params.rsi_overbought:
            self.log(f'Sell: %.2f' % (self.data[0]))
            self.close()  # close long position

In [25]:
# Initialize Engine
cerebro = bt.Cerebro()

# Initialize Broker Configs
cerebro.broker.setcash(1000000.0)  # Starting with 1 Lakh cash
# cerebro.broker.setcommission(0.0003)  # 0.03% Brokerage


# Feed Data
data0 = bt.feeds.PandasData(dataname=data, tz=pytz.timezone('Asia/Kolkata'))
cerebro.adddata(data0)

# Add Strategy
cerebro.addstrategy(RSI)

# Add Trade Analyzer
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name = "ta")

# Backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000000.00
2020-01-31T00:00:00: Buy : 11962.10
2020-02-03T00:00:00: BUY  Executed, 11627.45
2020-06-01T00:00:00: Sell: 9826.15
2020-06-02T00:00:00: SELL Executed, 9880.85
2020-06-02T00:00:00: Trade Summary, gross: -1746.60, net: -1746.60, duration: 78

2020-09-23T00:00:00: Buy : 11131.85
2020-09-24T00:00:00: BUY  Executed, 11011.00
2020-10-06T00:00:00: Sell: 11662.40
2020-10-07T00:00:00: SELL Executed, 11679.25
2020-10-07T00:00:00: Trade Summary, gross: 668.25, net: 668.25, duration: 8

2021-01-29T00:00:00: Buy : 13634.60
2021-02-01T00:00:00: BUY  Executed, 13758.60
2021-02-02T00:00:00: Sell: 14647.85
2021-02-03T00:00:00: SELL Executed, 14754.90
2021-02-03T00:00:00: Trade Summary, gross: 996.30, net: 996.30, duration: 2

2021-03-25T00:00:00: Buy : 14324.90
2021-03-26T00:00:00: BUY  Executed, 14506.30
2021-05-18T00:00:00: Sell: 15108.10
2021-05-19T00:00:00: SELL Executed, 15058.60
2021-05-19T00:00:00: Trade Summary, gross: 552.30, net: 552.30, duration: 33

202

## Strategy with Target and StopLoss

### MaruBozu Strategy

Marubozu is a candlestick with no upper and lower shadow.

- A bullish marubozu indicates bullishness.
    - Buy around the closing price of a bullish marubozu
    - Keep the low of the marubozu as the stoploss
- A bearish marubozu indicates bearishness.
    - Sell around the closing price of a bearish marubozu
    - Keep the high of the marubozu as the stoploss

In [26]:
class MarubozuStrategy(BaseStrategy):
    params = dict(
        rrr=2,  # risk reward ratio
    )

    def __init__(self):
        super().__init__()
        self.orefs = []

        # Marubozu indicator
        self.marubozu = bt.talib.CDLMARUBOZU(self.data.open, self.data.high, self.data.low, self.data.close)

    def notify_order(self, order):
        if not order.alive() and order.ref in self.orefs:
            self.orefs.remove(order.ref)

    def next(self):
        if self.orefs:
            return  # pending orders do nothing

        # Check if we are in the market
        if not self.position:
            # We are not in the market, look for a signal to OPEN trades
            if self.marubozu[0] == 100:
                # strong buy momentum, open long position
                stoploss = self.data.low[0]  # Low of marubozu will act as stoploss
                target = (self.params.rrr * (self.data.close[0] - stoploss)) + self.data[0]
                self.log(f'Long : %.2f, Target: %.2f, Stoploss: %.2f' % (self.data[0], target, stoploss))
                os = self.buy_bracket(price=self.data[0], limitprice=target, stopprice=stoploss)
                self.orefs = [o.ref for o in os]
            elif self.marubozu[0] == -100:
                # strong sell momentum, open short position
                stoploss = self.data_high[0]  # High of marubozu will act as stoploss
                target = self.data[0] - (self.params.rrr * (stoploss - self.data[0]))
                self.log(f'Short: %.2f, Target: %.2f, Stoploss: %.2f' % (self.data[0], target, stoploss))
                os = self.sell_bracket(price=self.data[0], limitprice=target, stopprice=stoploss)
                self.orefs = [o.ref for o in os]


## Appendix

### Logging DateTime with Timezone Support in Backtrader
When analyzing intraday trades, having accurate timestamps with timezone support can be crucial for understanding when trades occur in relation to market hours. Backtrader supports timezone handling, allowing you to log the exact datetime of each event during strategy execution.



In [27]:
import pytz

class BaseStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        """Logging function for strategy"""
        dt = dt or self.data.datetime.datetime(0)
        print(f'{dt.isoformat()}: {txt}')

# When feeding your data into Backtrader,
# you can specify the timezone using the tz parameter.
data0 = bt.feeds.PandasData(dataname=data, tz=pytz.timezone('Asia/Kolkata'))

### Simulating Broker Commision

Within the regular cerebro creation/set-up process, just add a call to setcommission over the broker member attribute.

In [28]:
# cerebro.broker.setcommission(0.0003)  # 0.03% Brokerage

To Log the commision related info use following:

In [29]:
# self.log("Trade Summary, gross: %.2f, net: %.2f, duration: %d\n" % (trade.pnl, trade.pnlcomm, trade.barlen))
# self.log("BUY EXECUTED, price: %.2f, commision: %.2f" % (order.executed.price, order.executed.comm))

### Customize Order Size

In [30]:
# cerebro.addsizer(bt.sizers.AllInSizerInt)
# cerebro.addsizer(bt.sizers.FixedSize, stake=10)